In [12]:
from math import comb

# ---------------------------
# Parameters
# ---------------------------
face_value = 100
coupon = 10
maturity = 6
delivery_time = 4
q = 0.5

# ---------------------------
# Short rate tree (from image)
# ---------------------------
short_rate_tree = [
    [0.06],
    [0.075, 0.054],
    [0.0938, 0.0675, 0.0486],
    [0.1172, 0.0844, 0.0608, 0.0437],
    [0.1465, 0.1055, 0.0759, 0.0547, 0.0394],
    [0.1831, 0.1318, 0.0949, 0.0683, 0.0492, 0.0354]
]

# ---------------------------
# Step 1: Build bond price tree using backward induction
# ---------------------------
def build_bond_tree(short_rate_tree, face_value, coupon, maturity):
    bond_tree = [[] for _ in range(maturity + 1)]
    bond_tree[maturity] = [face_value + coupon] * (maturity + 1)

    for t in range(maturity - 1, -1, -1):
        row = []
        for i in range(t + 1):
            r = short_rate_tree[t][i]
            up = bond_tree[t + 1][i]
            down = bond_tree[t + 1][i + 1]
            expected = q * up + (1 - q) * down
            price = coupon + expected / (1 + r)
            row.append(price)
        bond_tree[t] = row

    return bond_tree

bond_tree = build_bond_tree(short_rate_tree, face_value, coupon, maturity)

# ---------------------------
# Step 2: Get ex-coupon prices at t = 4
# ---------------------------
Z_4 = [price - coupon for price in bond_tree[delivery_time]]

# ---------------------------
# Step 3: Compute B_t (cash account) recursively
# ---------------------------
def build_cash_account_tree(short_rate_tree, delivery_time):
    B = [[1.0]]

    for t in range(1, delivery_time + 1):
        row = []
        for i in range(t + 1):
            if i == 0:
                val = B[t - 1][0] * (1 + short_rate_tree[t - 1][0])
            elif i == t:
                val = B[t - 1][t - 1] * (1 + short_rate_tree[t - 1][t - 1])
            else:
                val = B[t - 1][i - 1] * (1 + short_rate_tree[t - 1][i - 1])
            row.append(val)
        B.append(row)

    return B

B_tree = build_cash_account_tree(short_rate_tree, delivery_time)
B_4 = B_tree[delivery_time]

# ---------------------------
# Step 4: Compute G0 using forward pricing formula
# ---------------------------
def compute_forward_price(Z_4, B_4, q):
    numerator = 0.0
    denominator = 0.0

    for i in range(len(Z_4)):
        weight = comb(delivery_time, i) * (q ** i) * ((1 - q) ** (delivery_time - i))
        numerator += weight * (Z_4[i] / B_4[i])
        denominator += weight * (1 / B_4[i])

    return numerator / denominator

G_0 = compute_forward_price(Z_4, B_4, q)

# Helper to print a lattice in aligned form
def print_lattice(title, lattice, round_digits=4):
    print(f"\n🔷 {title}")
    max_width = len(lattice[-1])
    for t, row in enumerate(lattice):
        padding = " " * (max_width - len(row))
        formatted = [f"{val:.{round_digits}f}" for val in row]
        print(f"t={t}: {padding}{formatted}")

# Print the bond price lattice
print_lattice("Bond Price Lattice", bond_tree)

# Format Z_4 ex-coupon prices
print("\n📉 Ex-Coupon Prices at t=4 (Z_4^6):")
for i, z in enumerate(Z_4):
    print(f"Node {i}: Z_4 = {z:.4f}")

# Print the cash account lattice up to t=4
print_lattice("Cash Account Lattice (B_t up to t=4)", B_tree)

# Print final forward price
print(f"\n🎯 Final Forward Price G₀ at t=0: {G_0:.4f}")

q = 0.5  # risk-neutral probability
expiry = 4

# Revised version: Pricing a futures contract on an ex-coupon bond (delivered just after coupon at t=4)

# Step 1: Get the ex-coupon prices at t=4 (already known from previous calculation)
Z_4 = [price - coupon for price in bond_tree[4]]  # Ex-coupon bond prices

# Step 2: Rebuild the spot tree as if it's the ex-coupon bond being delivered
# We'll rebuild the spot tree from scratch using backward induction, but ending at Z_4

# Initialize new "ex-coupon spot price" tree
ex_coupon_tree = [[] for _ in range(5)]  # only up to t=4
ex_coupon_tree[4] = Z_4

# Backward induction to fill ex_coupon_tree from t=3 to t=0
for t in range(3, -1, -1):
    row = []
    for i in range(t + 1):
        r = short_rate_tree[t][i]
        up = ex_coupon_tree[t + 1][i]
        down = ex_coupon_tree[t + 1][i + 1]
        expected = q * up + (1 - q) * down
        price = expected / (1 + r)
        row.append(price)
    ex_coupon_tree[t] = row

# Step 3: Initialize futures tree with F_4 = Z_4
futures_tree = [[] for _ in range(5)]
futures_tree[4] = Z_4

# Step 4: Backward induction to compute futures prices using ex-coupon prices
for t in range(3, -1, -1):
    F_t = []
    for i in range(t + 1):
        # Use B_tree values from earlier
        B_up = B_tree[t + 1][i]
        B_down = B_tree[t + 1][i + 1]
        F_up = futures_tree[t + 1][i]
        F_down = futures_tree[t + 1][i + 1]

        # Risk-neutral expectation
        num = q * (F_up / B_up) + (1 - q) * (F_down / B_down)
        denom = q * (1 / B_up) + (1 - q) * (1 / B_down)
        F = num / denom
        F_t.append(F)
    futures_tree[t] = F_t

# Print spot price lattice (bond prices)
print_lattice("Spot Price Lattice (Underlying Bond)", ex_coupon_tree)

# Print futures price lattice
print_lattice("Futures Price Lattice (Contract Expiring at t=4)", futures_tree)



🔷 Bond Price Lattice
t=0:       ['124.1358']
t=1:      ['115.8271', '126.1408']
t=2:     ['108.9750', '118.5533', '126.2715']
t=3:    ['104.0288', '112.4889', '119.2725', '124.5721']
t=4:   ['101.6556', '108.4424', '113.8355', '117.9971', '121.1607']
t=5:  ['102.9761', '107.1903', '110.4658', '112.9673', '114.8418', '116.2391']
t=6: ['110.0000', '110.0000', '110.0000', '110.0000', '110.0000', '110.0000', '110.0000']

📉 Ex-Coupon Prices at t=4 (Z_4^6):
Node 0: Z_4 = 91.6556
Node 1: Z_4 = 98.4424
Node 2: Z_4 = 103.8355
Node 3: Z_4 = 107.9971
Node 4: Z_4 = 111.1607

🔷 Cash Account Lattice (B_t up to t=4)
t=0:     ['1.0000']
t=1:    ['1.0600', '1.0600']
t=2:   ['1.1395', '1.1395', '1.1172']
t=3:  ['1.2464', '1.2464', '1.2164', '1.1715']
t=4: ['1.3925', '1.3925', '1.3516', '1.2904', '1.2227']

🎯 Final Forward Price G₀ at t=0: 103.3879

🔷 Spot Price Lattice (Underlying Bond)
t=0:     ['79.8260']
t=1:    ['79.9890', '89.2421']
t=2:   ['81.5254', '90.4510', '97.6714']
t=3:  ['85.0779', '93.26